In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os

import wget

In [9]:
%run helperfunctions.py

db = db_name

In [2]:
data_url = 'https://www.worldcubeassociation.org/results/misc/WCA_export.tsv.zip'
zip_name = 'WCA_export.tsv.zip'
wget.download(data_url,zip_name)

'WCA_export.tsv.zip'

In [3]:
zfile = zipfile.ZipFile(zip_name)
zfile.extractall('data')

In [14]:
recreatedb(db)

In [15]:
file_path='./data/'
file_prefix = 'WCA_export_'
file_sufix = '.tsv'
for file_name in os.listdir('./data/'):
  if file_name not in ['metadata.json','README.md']:
    df_name = file_name.replace('WCA_export_','').replace('.tsv','')
    #locals()['df_'+df_name]=pd.read_csv(file_path+file_name,'\t')
    append_to_db(pd.read_csv(file_path+file_name,'\t'),df_name.lower(),db)

In [ ]:
sql = """
CREATE VIEW edges2018 AS
 WITH p AS (
         SELECT DISTINCT p.uid,
            r.competitionid
           FROM persons p
             JOIN results r ON r.personid = p.id
             JOIN competitions comp ON comp.id = r.competitionid
          WHERE comp.year = 2018
        )
 SELECT a.uid AS source,
    b.uid AS target,
    count(*) AS weight
   FROM p a
     JOIN p b ON a.competitionid = b.competitionid AND a.uid < b.uid
  GROUP BY a.uid, b.uid;
"""

execsql(sql,db)

In [ ]:
sql = """
CREATE VIEW nodes2018 AS
 SELECT DISTINCT p.uid AS id,
    c.continentid AS label
   FROM persons p
     JOIN countries c ON c.id = p.countryid
     JOIN results r ON r.personid = p.id
     JOIN competitions comp ON comp.id = r.competitionid
  WHERE comp.year = 2018;
;"""

execsql(sql,db)